## Descriptors:

 - __get__ -> p.x
 - __set__ -> p.x=100
 - __delete__ -> del p.x
 - __set_name__ -> for later
 
 
 For now, only non-data descriptors
 

In [5]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()
    
class Logger:
    current_time = TimeUTC()
    
l = Logger()
print(l.current_time)

'2025-04-20T01:46:19.730020'

In [8]:
from random import choice, seed


# without descriptors
class Deck:
    
    @property
    def suit(self):
        return choice(('Space', 'Heart', 'Diamond', 'Club'))
    
    @property
    def card(self):
        return choice(tuple('23456789JQKA')+('10', ))

d = Deck()
seed(0)

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Space
Q Diamond


In [14]:
# with descriptors

class Choice:
    def __init__(self, *choices):
        self.choices = choices
        
    def __get__(self, instance, owner_class):
        return choice(self.choices)
    
class Deck2:
    suit = Choice('Space', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JQK', '10')
    
d2 = Deck2()

for _ in range(10):
    print(d.card, d.suit)
    
class Dice:
    die1 = Choice(1, 2, 3, 4, 5, 6)
    die2 = Choice(1, 2, 3, 4, 5, 6)
    die3 = Choice(1, 2, 3, 4, 5, 6)
    
dice = Dice()
for _ in range(10):
    print(dice.die1, dice.die2, dice.die3)

7 Club
3 Diamond
J Diamond
A Space
J Diamond
J Heart
10 Diamond
9 Space
Q Club
7 Heart
3 2 2
2 1 5
6 3 4
1 1 6
2 2 1
1 6 5
6 4 6
5 3 5
2 2 6
5 4 5


In [21]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return datetime.utcnow().isoformat()
        
class Logger:
    current_time = TimeUTC()
    
Logger.current_time

l = Logger()
l.current_time

'2025-04-20T02:40:00.599780'

In [23]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f'__get__ called in {self}')
            return datetime.utcnow().isoformat()
        
class Logger:
    current_time = TimeUTC()
    
l1 = Logger()
l2 = Logger()

l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x7fa1b5fef940>
__get__ called in <__main__.TimeUTC object at 0x7fa1b5fef940>


('2025-04-20T02:42:45.789940', '2025-04-20T02:42:45.789952')

In [34]:
# __set__ method 


class Integervalue:
    def __set__(self, instance, value):
        print(f'__set__ called, instance ={instance}, value={value}')
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from class')
        else:
            print(f'__get__ called, instance ={instance}, owner_class={owner_class}')
            
class Point2d:
    x = Integervalue()
    y = Integervalue()

In [37]:
p = Point2d()
p.x

__get__ called, instance =<__main__.Point2d object at 0x7fa1b5cf9d90>, owner_class=<class '__main__.Point2d'>


In [38]:
p.x = 100

__set__ called, instance =<__main__.Point2d object at 0x7fa1b5cf9d90>, value=100


In [39]:
p2 = Point2d()
p2.x

__get__ called, instance =<__main__.Point2d object at 0x7fa1b5f94d00>, owner_class=<class '__main__.Point2d'>


In [41]:
p.x

__get__ called, instance =<__main__.Point2d object at 0x7fa1b5cf9d90>, owner_class=<class '__main__.Point2d'>


In [50]:
# with weak reference

import weakref

class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)
        
class Point:
    x = IntegerValue()
    
p = Point()
print(hex(id(p)))

p.x = 100.01
print(Point.x.values.keyrefs())

del p
print(Point.x.values.keyrefs())


0x7fa1b78c0c10
[<weakref at 0x7fa1b77cc3b0; to 'Point' at 0x7fa1b78c0c10>]
[]


In [55]:
# with weak reference

import weakref

class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))
        
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x
    
p = Point(10.1)
print(hex(id(p)))

p.x = 100.01
#print(Point.x.values.keyrefs())

del p
#print(Point.x.values.keyrefs())


0x7fa1b76afe20


In [58]:
# descriptor is a property


from numbers import Integral

class Person:
    
    @property
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be a non-negative integer')
        self._age = value
        

p = Person()
try:
    p.age = -10
except ValueError as ex:
    print(ex)
            
            
    
p.age = 10

p.__dict__

age: must be a non-negative integer


{'_age': 10}

In [ ]:
# Applicaiton

class Int:
    
    def __set_name__(name, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(calue, int):
            raise ValueError(f'{self.prop_name} must be an integer')
        instance.__dict__[self.prop_name] = name
    
    def __get(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
        
    
class Float:
    
    def __set_name__(name, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(calue, float):
            raise ValueError(f'{self.prop_name} must be an float')
        instance.__dict__[self.prop_name] = name
    
    def __get(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
        
        
class List:
    
    def __set_name__(name, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(calue, list):
            raise ValueError(f'{self.prop_name} must be an list')
        instance.__dict__[self.prop_name] = name
    
    def __get(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

#repetition...
#possible solution:

class ValidType:
    
    def __init__(self, type_):
        self._type = type_
        
    def __set_name__(name, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(calue, _type):
            raise ValueError(f'{self.prop_name} must be an {str(self._type.__name__)}')
        instance.__dict__[self.prop_name] = name
    
    def __get(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)